<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'>&copy; 2015 Karim Pichara - Christian Pieringer. Todos los derechos reservados. <br> 
   Editado por Equipo Docente IIC2233 2018-1, 2018-2, 2019-1, 2019-2, 2020-1 y 2021-1, y extendido con material creado en 2017-2 por Hugo Navarrete e Ignacio Acevedo.</font>
</p>

# Eventos y señales

Recordemos que las interfaces gráficas son aplicaciones construidas con una **arquitectura basada en manejo de eventos**. Esto significa que la aplicación detecta eventos generados por el usuario o por otras partes de la misma aplicación, y los procesa con el elemento apropiado. 

Una aplicación construida con PyQt empieza a detectar eventos una vez que ha entrado en su *mainloop*, y esto ocurre luego de llamar al método `exec()` de `QApplication`. La siguiente figura muestra una comparación, mediante diagramas de flujo, entre un programa con una estructura lineal como los que habíamos construido en las semanas anteriores, y un programa con uso de GUI basada en manejo de eventos.

![](img/gui-flowchart.png)

En este modelo existen 3 elementos fundamentales:

- **La fuente del evento**: Corresponde al objeto que genera el cambio de estado o que genera el evento
- **El objeto evento**: Es el objeto que encapsula el cambio de estado mediante el evento. 
- **El objeto destino:** Equivale al objeto al que se le desea notificar del cambio de estado

Bajo este modelamiento, la **fuente del evento** delega la tarea de manejar el evento al **objeto destino** pasándole el **objeto evento**. Para implementar este modelo, PyQt utiliza un mecanismo de ***signal*** y ***slot*** que le permite manejar los eventos y hacer posible la comunicación entre *widgets*. De esta forma, cuando un evento ocurre, el objeto que es activado emite una señal (*signal*) a la ranura (*slot*) correspondiente, donde un *slot* puede ser cualquier tipo de elemento llamable (*callable*) en Python.

En el siguiente ejemplo, extenderemos el programa anterior que mostraba botones dispuestos en grilla que simulaban una calculadora, para generar una llamada a la función `boton_clickeado` cada vez que se presione alguno de los botones. Esto se logra conectando el evento que enviará la señal con el *slot* que la recibe. En el caso de los botones, generalmente el método corresponde al evento `clicked`. Mediante el método `connect()` se establece la comunicación entre el evento y el *slot*. Este método recibe una función *llamable* en Python, (recordemos no agregar los `()`, ya que con esa sentencia estaríamos entregándole el resultado de la función y no el objeto función). El  ejemplo solo muestra la clase `MiVentana`.

In [1]:
import sys
from PyQt5.QtWidgets import (QApplication, QWidget, QPushButton, QLabel,
                             QLineEdit, QGridLayout, QHBoxLayout)


class MiVentana(QWidget):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.init_gui()

    def init_gui(self):
        self.label1 = QLabel('Status:', self)
        self.grilla = QGridLayout()

        valores = ['1', '2', '3',
                   '4', '5', '6',
                   '7', '8', '9',
                   '0', 'CE', 'C']

        posiciones = [(i, j) for i in range(4) for j in range(3)]
        
        for posicion, valor in zip(posiciones, valores):
            boton = QPushButton(valor, self)

            """
            Aquí conectamos el evento clicked() de cada boton con el slot
            correspondiente. En este ejemplo todos los botones usan el
            mismo slot (self.boton_clickeado).
            """
            boton.clicked.connect(self.boton_clickeado)  # Connect recibe función que se ejecutará al hacer click

            self.grilla.addWidget(boton, *posicion)

        vbox = QHBoxLayout()
        vbox.addWidget(self.label1)
        vbox.addLayout(self.grilla)
        self.setLayout(vbox)

        self.move(300, 150)
        self.setWindowTitle('Calculator')

    def boton_clickeado(self):
        """
        Esta función se ejecuta cada vez que uno de los botones de la grilla
        es presionado. Cada vez que el botón genera el evento, la función inspecciona
        cual botón fue presionado y recupera la posición en que se encuentra en
        la grilla.
        """

        # Sender retorna el objeto que fue clickeado.
        # Ahora, la variable boton referencia una instancia de QPushButton
        boton = self.sender()

        # Obtenemos el identificador del elemento en la grilla
        idx = self.grilla.indexOf(boton)

        # Con el identificador obtenemos la posición del ítem en la grilla
        posicion = self.grilla.getItemPosition(idx)

        # Actualizamos label1
        self.label1.setText(f'Status: Presionado boton {idx}, en fila/columna: {posicion[:2]}.')


if __name__ == '__main__':
    app = QApplication([])
    form = MiVentana()
    form.show()
    sys.exit(app.exec())

SystemExit: 0

c:\Users\benja\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


**Nota sobre `label.setText`:** En algunos sistemas operativos, la actualización de *labels* con `setText` no se refleja inmediatamente como se esperaría. Si esto ocurre, hay que ejecutar el método `repaint` de `QLabel` para forzar la actualización, como se muestra a continuación:

```python
# label es una instancia de QLabel
label.setText("Nuevo texto")
label.repaint()
```


**Realiza el ejercicio propuesto 3.1 para experimentar con el uso de eventos de botones**

## Obtener al emisor de la señal: `sender`

En ocasiones, como en la función `boton_clickeado()` del código anterior, es necesario conocer cuál de los objetos del formulario envió una señal. El método `sender()` retorna el objeto que generó el evento. El siguiente ejemplo utiliza `sender()` para obtener una referencia al objeto que generó el evento y mostrar cuál objeto generó el evento. 

En el mismo ejemplo, puedes notar que se usa la clase `QCoreApplication`, de `QtCore`, la que permite obtener una referencia al objeto `QApplication` que está en ejecución. Esto se utiliza para forzar una salida de la ventana al hacer *click* en el *slot* del tercer botón.

In [1]:
import sys
from PyQt5.QtWidgets import (QApplication, QWidget, QPushButton, QLabel, QHBoxLayout, QVBoxLayout)
from PyQt5.QtCore import QCoreApplication


class MiVentana(QWidget):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.init_gui()

    def init_gui(self):
        self.label1 = QLabel('Status: -', self)

        """
        El evento de cada botón es conectado con su slot. En este caso es 
        el mismo método boton_callback().
        """
        self.boton1 = QPushButton('&Boton 1', self)
        self.boton1.resize(self.boton1.sizeHint())
        self.boton1.clicked.connect(self.boton_clickeado)

        self.boton2 = QPushButton('&Boton 2', self)
        self.boton2.resize(self.boton2.sizeHint())
        self.boton2.clicked.connect(self.boton_clickeado)

        self.boton3 = QPushButton('&Salir', self)
        self.boton3.resize(self.boton3.sizeHint())
        self.boton3.clicked.connect(QCoreApplication.instance().quit)

        hbox = QHBoxLayout()
        hbox.addStretch(1)
        hbox.addWidget(self.boton1)
        hbox.addWidget(self.boton2)
        hbox.addWidget(self.boton3)
        hbox.addStretch(1)

        vbox = QVBoxLayout()
        vbox.addStretch(1)
        vbox.addWidget(self.label1)
        vbox.addLayout(hbox)
        vbox.addStretch(1)
        self.setLayout(vbox)

        # Agregamos todos los elementos al formulario
        self.setGeometry(200, 100, 300, 200)
        self.setWindowTitle('Sender')

    def boton_clickeado(self):
        # Esta función registra el objeto que envía la señal del evento
        # y lo refleja mediante el método sender() en label3.
        sender = self.sender()
        self.label1.setText(f'Status: presionado botón {sender.text()}')
        self.label1.resize(self.label1.sizeHint())


if __name__ == '__main__':
    app = QApplication([])
    form = MiVentana()
    form.show()
    sys.exit(app.exec())


SystemExit: 0

c:\Users\benja\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


: 

## Eventos de *mouse*

### *Press Event*

No solo los botones tienen implementados eventos de interacción con el usuario: todo *widget* tiene distintos gatillos de eventos que se pueden conectar a *slots*. Por ejemplo, la clase `QWidget` incluye el método `mousePressEvent`, que se hace cargo del comportamiento del programa al presionar el *mouse* sobre el *widget* que lo implementa. Por defecto, este método no hace nada, pero siempre podemos hacer *override* de él en nuestros propios *widgets*. Similar a como `QPushButton` tenía una palabra reservada para el evento `clicked` para el cual podía vincular un método o función, `mousePressEvent` refleja el evento genérico de un clic para cualquier *widget*. Pero, la diferencia es que no es necesario realizar la conexión evento-método (`.clicked.connect(...)`), para este tipo de eventos (y los siguientes ejemplos) la conexión ya está definida y es solo necesario sobreescribir el método correspondiente.

Es decir, al sobrescribir el método `mousePressEvent` de un `QWidget` se está definiendo el código que se ejecutará una vez que se haga clic sobre ese *widget*.

En el siguiente ejemplo, hay dos *labels* de colores dentro del *widget*. Cuando se haga clic sobre él, dependiendo de la posición presionada, se ocultará o mostrará el *label* correspondiente.

In [1]:
import sys
import os

from PyQt5.QtWidgets import (QApplication, QWidget, QPushButton, QLabel)
from PyQt5.QtGui import QPixmap
from PyQt5.QtCore import QRect


class MiVentana(QWidget):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.init_gui()

    def init_gui(self):
        self.setGeometry(300, 100, 225, 450)
        self.setMaximumHeight(450)
        self.setMaximumWidth(225)
        self.setWindowTitle('mousePressEvent')

        self.mostrar_azul = True
        self.label_azul = QLabel('AZUL', self)
        self.label_azul.move(0, 0)
        self.label_azul.setGeometry(QRect(0, 0, 225, 225))  # (x, y, height, width)
        ruta_imagen_azul = os.path.join('img', 'colors', 'azul.png')
        self.pixmap_azul = QPixmap(ruta_imagen_azul)
        self.label_azul.setPixmap(self.pixmap_azul)
        self.label_azul.show()

        self.mostrar_verde = True
        self.label_verde = QLabel('VERDE', self)
        self.label_verde.move(0, 0)
        self.label_verde.setGeometry(QRect(0, 225, 225, 225))
        ruta_imagen_verde = os.path.join('img', 'colors', 'verde.png')
        self.pixmap_verde = QPixmap(ruta_imagen_verde)
        self.label_verde.setPixmap(self.pixmap_verde)
        self.label_verde.show()

        self.show()

    def mousePressEvent(self, event):
        if event.y() <= 225:  # Este es el alto del label
            if self.mostrar_azul:
                self.label_azul.hide()
            else:
                self.label_azul.show()
                # algo que pude haber escrito dos veces pero puede ser escrito solamente una vez
            self.mostrar_azul = not self.mostrar_azul
        else:
            if self.mostrar_verde:
                self.label_verde.hide()
            else:
                self.label_verde.show()
            self.mostrar_verde = not self.mostrar_verde


if __name__ == '__main__':
    app = QApplication([])
    form = MiVentana()
    sys.exit(app.exec())


SystemExit: 0

c:\Users\benja\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


**Realiza el ejercicio propuesto 3.2 para experimentar con el funcionamiento del método `mousePressEvent`**

### *Move Event*

Otro aspecto en que nuestro programa puede estar pendiente del usuario, es en revisar la posición en la que se encuentra el *mouse* **cuando este está sobre el *widget* (sin necesariamente presionar)**. 

En el siguiente ejemplo, vamos a ver la posición global del *mouse* mediante el evento `mouseMoveEvent`. Para esto, vamos a activar el *tracking* del *mouse* en el `widget` y `label` que queramos seguir. En el ejemplo, solo lo activamos en el *label* azul; el *label* verde no mostrará la posición del *mouse* en consola.

In [1]:
import sys
import os
from PyQt5.QtWidgets import (QApplication, QWidget, QLabel)
from PyQt5.QtGui import QPixmap
from PyQt5.QtCore import QRect


class MiVentana(QWidget):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.init_gui()

    def init_gui(self):
        self.setGeometry(300, 100, 225, 450)
        self.setMaximumHeight(450)
        self.setMaximumWidth(225)
        self.setWindowTitle('Move Event')
        
        # Creamos el label
        self.label_azul = QLabel('AZUL', self)
        self.label_azul.move(0, 0)
        self.label_azul.setGeometry(QRect(0, 0, 225, 225))  # (x, y, height, width)
        
        self.label_verde = QLabel('VERDE', self)
        self.label_verde.move(0, 0)
        self.label_verde.setGeometry(QRect(0, 225, 225, 225))
        
        ruta_imagen_azul = os.path.join('img' ,'colors', 'azul.png')
        self.pixmap_azul = QPixmap(ruta_imagen_azul) # Creamos el pixmap
        ruta_imagen_verde = os.path.join('img' ,'colors', 'verde.png')
        self.pixmap_verde = QPixmap(ruta_imagen_verde)
        
        self.label_azul.setPixmap(self.pixmap_azul) # Asignamos el pixmap
        self.label_verde.setPixmap(self.pixmap_verde)
        
        self.setMouseTracking(True) # Activamos el tracking en nuestra ventana
        self.label_azul.setMouseTracking(True)
        self.label_azul.show()
        self.label_verde.show()
        self.show()

    def mouseMoveEvent(self, event):
        objeto_posicion = event.pos()
        print(objeto_posicion.x(), objeto_posicion.y()) 


if __name__ == '__main__':
    app = QApplication([])
    form = MiVentana()
    sys.exit(app.exec())

214 107
208 98
204 92
199 87
195 84
192 82
189 80
186 78
184 76
182 76
179 75
177 74
175 73
173 72
170 72
169 72
166 72
164 72
162 72
160 72
157 72
156 72
154 72
153 72
152 72
151 72
150 72
149 72
148 72
147 72
147 73
147 74
147 75
147 76
147 77
147 78
147 79
147 80
147 81
147 82
147 83
147 84
147 85
147 86
147 87
147 88
147 90
147 91
147 94
146 97
146 100
145 104
145 109
145 113
144 118
143 124
143 127
142 131
142 135
141 138
141 140
140 143
139 146
139 149
138 151
137 155
137 159
136 162
135 164
134 167
133 170
132 173
132 174
132 175
132 176
132 177
132 178
131 179
131 180
130 180
130 181
130 182
130 183
130 184
129 185
129 186
128 187
128 188
128 189
128 190
128 191
127 191
127 192
127 193
127 194
127 195
127 196
127 197
127 198
127 199
127 200
127 201
127 202
127 203
127 204
127 205
127 207
126 209
126 210
125 214
124 217
123 222
218 224
218 219
218 216
218 212
219 209
219 206
220 204
220 201
220 199
220 197
221 195
221 192
221 191
221 188
221 184
222 182
222 179
222 175
222 172
2

SystemExit: 0

c:\Users\benja\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Existen muchos más eventos relacionados con el mouse que puedes revisar explorando la documentación de PyQt en la web.

## Eventos del teclado

Otra de las maneras más comunes en que una interfaz gráfica genera eventos por parte del usuario,
es a través del teclado. Cuando un usuario **presiona una tecla** ocurren eventos para los que muchas veces queremos definir un comportamiento. La clase `QWidget` incluye el método `keyPressEvent`, que se hace cargo del comportamiento del programa ante la presión de una tecla. Por defecto, este método no hace nada, pero siempre podemos hacer *override* de el en nuestros propios *widgets*.

In [1]:
import sys
from PyQt5.QtWidgets import (QApplication, QWidget, QLabel)


class MiVentana(QWidget):
    def __init__(self):
        super().__init__()
        self.inicializa_gui()

    def inicializa_gui(self):
        self.etiqueta = QLabel('Etiqueta', self)
        self.etiqueta.move(20, 10)
        self.resize(self.etiqueta.sizeHint())

        self.setGeometry(300, 300, 290, 150)
        self.setWindowTitle('Teclado')
        self.show()

    def keyPressEvent(self, event):
        """
        Este método maneja el evento que se produce al presionar las teclas.
        """
        self.etiqueta.setText(f'Presionaron la tecla: {event.text()} de código: {event.key()}')
        self.etiqueta.resize(self.etiqueta.sizeHint())


if __name__ == '__main__':
    app = QApplication([])
    ex = MiVentana()
    sys.exit(app.exec())


SystemExit: 0

c:\Users\benja\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


**Realiza el ejercicio propuesto 3.3, donde podrás poner en práctica la definición y ejecución del método `keyPressEvent` para definir comportamientos ante eventos del teclado**

## Generar señales personalizadas

PyQt permite definir señales personalizadas por el programador, además de los predeterminados sobre el *mouse* y teclado. Para esto se debe crear un objeto que alojará (como atributo) la nueva señal, el que debe ser subclase de `QtCore.QObject`. Dentro de la subclase se crea la nueva señal como una instancia de `QtCore.pyqtSignal`. Finalmente, en los *widgets* involucrados se deben recibir las señales y conectar las funciones que manejan la señal.

- Mediante el método `emit` de `pyqtSignal` se emite la señal (indicando que ocurrió un evento).
- Mediante el método `connect`, también de `pyqtSignal`, se define la función o método a ejecutar cuando la señal es emitida.

El siguiente ejemplo muestra cómo generar una nueva señal entre dos ventanas separadas. La señal se activa al presionar alguna parte de la ventana izquierda, y su efecto es editar el contenido de la ventana derecha.

La señal se crea como un atributo de clase dentro de la clase `VentanaPresionable`, que es quien **emite el evento** cada vez que se presiona la ventana, se **conecta** en el *main* a uno de los métodos de `VentanaQueSeEdita`, y finalmente `VentanaQueSeEdita` es quien **recibe el evento** y edita su contenido cada vez que se emite la señal.

In [1]:
import sys
from PyQt5.QtCore import (pyqtSignal)
from PyQt5.QtWidgets import (QApplication, QWidget, QLabel)


class VentanaPresionable(QWidget):
    senal_escribir = pyqtSignal()

    def __init__(self):
        super().__init__()
        self.inicializa_gui()

    def inicializa_gui(self):
        self.etiqueta = QLabel('Presiona esta ventana', self)
        self.etiqueta.move(20, 10)
        self.etiqueta.resize(self.etiqueta.sizeHint())

        self.setGeometry(300, 300, 290, 150)
        self.setWindowTitle('Emite señal')
        self.show()

    def mousePressEvent(self, event):
        # Al ejecutar la siguiente línea, se emite la señal,
        # y los métodos conectados se llamarán automáticamente.
        self.senal_escribir.emit()


class VentanaQueSeEdita(QWidget):

    def __init__(self):
        super().__init__()
        self.inicializa_gui()

    def inicializa_gui(self):
        self.etiqueta = QLabel('', self)
        self.etiqueta.move(20, 10)
        self.etiqueta.resize(self.etiqueta.sizeHint())

        self.setGeometry(700, 300, 290, 150)
        self.setWindowTitle('Recibe señal')
        self.show()
    
    # Este es el método que conectaremos a la señal
    def edita_etiqueta(self):
        self.etiqueta.setText('¡Oh! Alguien ha presionado el mouse')
        self.etiqueta.resize(self.etiqueta.sizeHint())


if __name__ == '__main__':
    app = QApplication([])
    ventana_click = VentanaPresionable()
    ventana_edit = VentanaQueSeEdita()

    # Conectamos la señal
    ventana_click.senal_escribir.connect(ventana_edit.edita_etiqueta)
    # en resumen es: Ventana que tiene la señal -> señal que quiero conectar -> .connect
    # -> dentro de él método que se quiere conectar.

    sys.exit(app.exec())

SystemExit: 0

c:\Users\benja\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


#### (Abre paréntesis...

1. ¿Por qué se encapsula la instancia `pyqtSignal` dentro de una clase? ¿No se puede instanciar directamente y trabajar con el objeto `pyqtSignal`?
2. ¿Por qué se define como un atributo de clase y no de instancia?
3. ¿Esto último significa que todas las instancias de la clase `VentanaPresionable` compartirán la misma señal?

¡Muy buenas preguntas! Las tres prácticamente se responden de la misma forma: la razón de fondo es la implementación interna de PyQt que requiere estas condiciones. Ahora, específicamente:

1. Las `pyqtSignal` deben estar encapsulada por objetos que hereden de `QObject`. Se pueden instanciar señales fuera de una clase, pero estas no serán capaces de conectarse a funciones o ser emitidas por alguna entidad. ¿Por qué entonces podemos encapsular la señal en `VentanaPresionable`, que no es un `QObject`? Esto es simplemente porque nuestras ventanas heredan de `QWidget` y todo `QWidget` es a su vez un `QObject`.
2. Las instancias de `pyqtSignal` deben ser definidas como atributo de clase y no de instancia por diseño interno de PyQt. Puedes hacer la prueba e instanciarla como un atributo de instancia, y verás que esta no será capaz de conectarse a llamables o de emitirse.
3. Hemos visto que el comportamiento de un atributo de clase es distinto para un atributo de instancia para objetos mutables. En general, si tenemos un atributo de clase mutable (como una lista), si cualquier instancia modifica este atributo, todas las instancias tienen acceso a este atributo y ven el efecto. Entonces, ¿todas las instancias de `MiSenal` compartirían la señal `senal`? Soprendentemente, la respuesta es **no**. Esto se debe a implementación interna de PyQt que logra que a pesar de que se define a nivel de atributo de clase, para cada instancia la señal creada como atributo es única y distinta. Entonces, a pesar de que se defina a ese nivel, podemos estar tranquilos en que cada instancia tendrá su propia señal única.

#### ...cierra paréntesis)

### Emisión de eventos con información

En nuestro primer ejemplo básico de señal personalizada, se emitió y recibió un evento que simplemente ocurría y realizaba un efecto. Este no transmitía más información ni datos que describieran el evento que ocurrió. `pyqtSignal` permite una forma para que, al momento de emitir el evento (`emit`), se envíe también información a la función o método conectado y lo reciba como un argumento. Al instanciar la señal `pyqtSignal`, este recibe como argumentos los tipos de objetos que es capaz de enviar mediante `emit` y que recibirán los objetos llamables como argumentos. En el siguiente ejemplo se expande el anterior con tres señales separadas que emiten distinta información:

In [ ]:
import sys
from PyQt5.QtCore import (QObject, pyqtSignal)
from PyQt5.QtWidgets import (QApplication, QWidget, QLabel)


class VentanaPresionable(QWidget):
    senal_simple = pyqtSignal()  # Señal simple
    senal_texto = pyqtSignal(str)  # Señal que permite enviar texto
    senal_coordenadas = pyqtSignal(int, int)  # Señal que permite enviar dos ints

    def __init__(self):
        super().__init__()
        self.inicializa_gui()

    def inicializa_gui(self):
        self.etiqueta = QLabel('Texto etiqueta', self)
        self.etiqueta.move(20, 10)
        self.etiqueta.resize(self.etiqueta.sizeHint())

        self.setGeometry(300, 300, 290, 150)
        self.setWindowTitle('Emite señal')
        self.setMouseTracking(True)
        self.show()
    
    ### emitir señales dentro de los métodos como keypress event or mousepress event
    ### LOS EVENTOS EN LA VENTANA DE SALIDA EMITEN SEÑALES.
    ### LOS METODOS DENTRO DE LA VENTANA DE LLEGADA EJECUTAN ACCIONES.

    def mousePressEvent(self, event):
        # Se emite la señal simple, sin argumento
        self.senal_simple.emit()
        # Se emite la señal que permite envir un str
        # Se envia el contenido de la etiqueta de la ventana
        self.senal_texto.emit(self.etiqueta.text())

    def mouseMoveEvent(self, event):
        # Se emite la señal que permite enviar dos ints, enviamos la posición del mouse
        self.senal_coordenadas.emit(event.pos().x(), event.pos().y())


class VentanaQueSeEdita(QWidget):

    def __init__(self):
        super().__init__()
        self.inicializa_gui()

    def inicializa_gui(self):

        self.etiqueta_1 = QLabel('', self)
        self.etiqueta_1.move(20, 10)
        self.etiqueta_1.resize(self.etiqueta_1.sizeHint())

        self.etiqueta_2 = QLabel('', self)
        self.etiqueta_2.move(20, 40)
        self.etiqueta_2.resize(self.etiqueta_2.sizeHint())

        self.etiqueta_3 = QLabel('', self)
        self.etiqueta_3.move(20, 70)
        self.etiqueta_3.resize(self.etiqueta_3.sizeHint())

        self.setGeometry(700, 300, 290, 150)
        self.setWindowTitle('Recibe señal')
        self.show()
    
    def edita_etiqueta_click(self):
        # Este método no tiene argumentos, ya que se conectará a una señal simple
        self.etiqueta_1.setText('¡Oh! Alguien ha presionado el mouse')
        self.etiqueta_1.resize(self.etiqueta_1.sizeHint())

    def edita_etiqueta_texto(self, texto):
        # Este método tiene un argumento, el str que se espera del evento conectado
        self.etiqueta_2.setText(f'Recibí del evento: {texto}')
        self.etiqueta_2.resize(self.etiqueta_2.sizeHint())

    def edita_etiqueta_posicion_mouse(self, x, y):
        # Este método tiene dos argumentos, los ints que se espera del evento conectado
        self.etiqueta_3.setText(f'Recibí posiciones: {x}, {y}')
        self.etiqueta_3.resize(self.etiqueta_3.sizeHint())


if __name__ == '__main__':
    app = QApplication([])
    # senales = MisSenales()
    ventana_click = VentanaPresionable()
    ventana_edit = VentanaQueSeEdita()

    # Conectamos las señales
    ventana_click.senal_simple.connect(ventana_edit.edita_etiqueta_click)
    ventana_click.senal_texto.connect(ventana_edit.edita_etiqueta_texto)
    ventana_click.senal_coordenadas.connect(ventana_edit.edita_etiqueta_posicion_mouse)

    sys.exit(app.exec())